In [26]:
from os import listdir
import random
import pandas as pd
import math
import json
import operator
import logging
from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np
import json
# %from nltk import wordnet
from os import listdir
from os import mkdir
import random
import math
import json
import operator
import sys
import string
import time
def search(es_object, index_name, search):
    res = es_object.search(index=index_name, body=search, request_timeout = 60)
    return res

def connect_elasticsearch():
    _es = None
#     _es = Elasticsearch([{"host":"d5hadoop29.mpi-inf.mpg.de", "port": 9200}])
    _es = Elasticsearch([{"host":"d5hadoop22.mpi-inf.mpg.de", "client.transport.sniff" : True, "port": 9200}])
    return _es
informative_entity_types = set(["dsyn", "patf", "sosy", "dora", "fndg", "menp", "chem", "orch", "horm", "phsu", "medd", "bhvr", "diap", 'bacs','chem', 'enzy', "inpo", "elii"])
uninformative_entity_types = set(["phpr", "npop", 'bsoj', 'idcn',"sbst", "food", "evnt", "geoa", "idcn"])

def entity_info(es_object, index_name, entity):
    res = es_object.search(index=index_name, size=1, search_type="dfs_query_then_fetch", _source_includes=["human_readable", "types"], body={"filter": { "bool" : {  "must" : [{ "term" : {"kb_id" : entity}}, { "term" : {"_type" : "entity"}} ]}}})
    print(res)
    return res["hits"]["hits"][0]["_source"]


def is_informative_entity(es, entity = ""):
    ei = entity_info(es, "health-kb", entity)
    types = []
    if("types" in ei.keys()):
        types = [str(x) for x in ei["types"]]
    return is_informative_types(types)

# def is_informative_types(types = []):
#     return len(set(types).intersection(informative_entity_types)) > 0
def is_informative_types(types = []):
    return len(set(types).intersection(informative_entity_types)) > 0 or len([x for x in types if x.startswith("disease_affecting") or x.startswith("symptoms")]) > 0
def is_informative_type(t):
    return len(t) >4 and not t.startswith("interactions")
def count_co_occurrences(es, e1, e2):
    res = es.count(index="health-threads", body={"query": {"bool": {"must": [ { "term" : { "aida.allEntities" : e1 } },{ "term" : { "aida.allEntities" : e2}}, {"regexp": {"FeedUrl":".*(healthboards|ehealthforum|patient.co).*"}}]}}})
    return res[u'count']

def count_occurrences(es, entity):
    res = es.count(index="health-threads", body={"query": {"bool": {"must":[{ "term" : { "aida.allEntities" : entity }}, {"regexp": {"FeedUrl":".*(healthboards|ehealthforum|patient.co).*"}}]}}})
    return res[u'count']

# def count_all(es, entity):
#     res = es.count(index="health-docs", body={"query" : {"constant_score" : { "filter" : { "exists" : { "field" :"allEntities"}}}}})
#     return res[u'count']

def pmi(es, e1, e2, e1_occ, e2_occ, variant=2):
    cocc = float(count_co_occurrences(es, e1, e2))
    all_occ = float(1048428)
    if cocc == 0:
        pmi_uv =  -100
    else:
        pmi = math.log(cocc) - math.log(e1_occ) - (math.log(e2_occ) - math.log(all_occ))
        if(variant == 2):
            pmi_uv =  round(pmi + (math.log(cocc) - math.log(all_occ)),3)
    return cocc, pmi_uv

def get_types(es, entity):
    ei = entity_info(es, "health-kb", entity)
    types = []
    if("types" in ei.keys()):
        types = [str(x) for x in ei["types"]]
    return types
def compute_tf(es, entity, subforum):
    query = {"size":100000,"_source": ["aida.allMentionEntities"], "query": {"bool": {"must":[{"match": {"aida.allMentionEntities": entity}},{ "match_phrase": {"Description": subforum}},{ "match_phrase": {"FeedUrl": "http://ehealthforum.com/"}}]}}}
    res = search(es,"health-docs-v2", query)
    tf = 0
    for hit in res["hits"]["hits"]:
        hit_id = hit["_id"]
        tf += hit["_source"]["aida"]["allMentionEntities"].split(" ").count(entity)
    return tf
def compute_idf(es, entity):
    res = search(es, "health-docs-v2", {"size": 0,"_source": "Description", "query": {"bool": {"must":[{ "match": {"FeedUrl": "http://ehealthforum.com/"}},{"match": {"aida.allMentionEntities": entity}}]}},"aggs" : {"subforum_frequency" : {"cardinality" : {"field" : "Description"}}}})
    return res["aggregations"]["subforum_frequency"]["value"]
def unique_subforums(es):
    res = search(es, "health-docs-v2", {"size":300000,"_source": "Description", "query": {"bool": {"must": [{"match": {"FeedUrl": "http://ehealthforum.com/"}}]}}})
    sub_dic = {}
    for hit in res["hits"]["hits"]:
        [parent,forum] = hit["_source"]["Description"].split("\n-")
        sub_dic[forum] = parent+"\n-"+forum
    return sub_dic
def get_mayoclinic_encylopedia(es):
    res = search(es, "health-docs-v2", {"from" : 0, "size" : 3100, "_source": ["Title","aida.annotatedText"],    "query" : { "filtered" : { "query": { "bool": { "must" : [{"regexp": {"FeedUrl": ".*mayoclinic.*"}}]}}}}})
    ency_dic = {}
    for hit in res["hits"]["hits"]:
        title = hit["_source"]["Title"]
        try:
            text = ''.join(hit["_source"]["aida"]["annotatedText"].split("\n\n")[3:5])
            ency_dic[title] = text
        except:
            pass
    return ency_dic
es = connect_elasticsearch()

print(get_types(es, "C0037199"))
# is_informative_entity(es, "C0037199")

{'took': 32, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'failed': 0}, 'hits': {'total': 1, 'max_score': 1.0, 'hits': [{'_index': 'health-kb', '_type': 'entity', '_id': '19421', '_score': 1.0, '_source': {'types': ['causes_of_C0029443', 'causes_of_C0009450', 'dsyn', 'causes_of_C0012569', 'causes_of_C0238454', 'phpr', 'npop', 'disease_affecting_C0030471', 'causes_of_C0000833', 'complication_of_C0021400', 'patf', 'evnt', 'causes_of_C0262174', 'causes_of_C1260880', 'causes_of_C0003126', 'C0037199', 'risk_factors_for_C0031350', 'causes_of_C0009443', 'risk_factors_for_C0004096', 'symptoms_ofC0042769', 'complication_of_C3539909', 'risk_factors_for_C0149507', 'causes_of_C0234242', 'symptoms_ofC0017168', 'biof', 'causes_of_C0700148', 'symptoms_ofC0699744', 'causes_of_C0018681', 'causes_of_C0027424', 'complication_of_C0009443', 'risk_factors_for_C0035455'], 'human_readable': 'Sinusitis'}}]}}
['causes_of_C0029443', 'causes_of_C0009450', 'dsyn', 'causes_of_C0012569', 'causes_of_C

In [27]:
print(is_informative_entity(es, "C0037199"))


{'took': 34, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'failed': 0}, 'hits': {'total': 1, 'max_score': 1.0, 'hits': [{'_index': 'health-kb', '_type': 'entity', '_id': '19421', '_score': 1.0, '_source': {'types': ['causes_of_C0029443', 'causes_of_C0009450', 'dsyn', 'causes_of_C0012569', 'causes_of_C0238454', 'phpr', 'npop', 'disease_affecting_C0030471', 'causes_of_C0000833', 'complication_of_C0021400', 'patf', 'evnt', 'causes_of_C0262174', 'causes_of_C1260880', 'causes_of_C0003126', 'C0037199', 'risk_factors_for_C0031350', 'causes_of_C0009443', 'risk_factors_for_C0004096', 'symptoms_ofC0042769', 'complication_of_C3539909', 'risk_factors_for_C0149507', 'causes_of_C0234242', 'symptoms_ofC0017168', 'biof', 'causes_of_C0700148', 'symptoms_ofC0699744', 'causes_of_C0018681', 'causes_of_C0027424', 'complication_of_C0009443', 'risk_factors_for_C0035455'], 'human_readable': 'Sinusitis'}}]}}
True
